# **Import Necessory libraries**

In [8]:
import pandas as pd
from PIL import Image
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet101

# **Partial Cross_Entropy_Loss**

In [9]:
class PartialCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(PartialCrossEntropyLoss, self).__init__()

    def forward(self, predictions, targets, mask):
        focal_loss = F.cross_entropy(predictions, targets, reduction='none')
        masked_focal_loss = focal_loss * mask
        pfce_loss = masked_focal_loss.sum() / mask.sum()
        return pfce_loss


# **Load, Pre_process Data and Simulate Point Annotations**

In [18]:
class ForestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'image', self.data_frame.iloc[idx, 0])
        mask_name = os.path.join(self.root_dir, 'mask', self.data_frame.iloc[idx, 1])

        # Print the paths for debugging
        print(f"Image path: {img_name}")
        print(f"Mask path: {mask_name}")

        image = Image.open(img_name).convert("RGB")
        mask = Image.open(mask_name).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        mask_array = np.array(mask)
        point_mask = np.zeros_like(mask_array)
        points = random.sample(range(mask_array.size), k=100)  # Randomly sample 100 points
        for point in points:
            point_mask[np.unravel_index(point, mask_array.shape)] = mask_array[np.unravel_index(point, mask_array.shape)]

        return image, torch.tensor(mask_array, dtype=torch.long), torch.tensor(point_mask, dtype=torch.long)

# Define transformations for the images and masks
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Set the paths to your dataset
csv_file_path = '/content/drive/MyDrive/Forest Aerial Segmentation/meta_data.csv'
root_directory = '/content/drive/MyDrive/Forest Aerial Segmentation'

# Create dataset and dataloader
dataset = ForestDataset(csv_file=csv_file_path, root_dir=root_directory, transform=transform)

# Check the length of the dataset
print(f"Total items in dataset: {len(dataset)}")


Total items in dataset: 368


# **Custom Loss Function and Segmentation Network**

In [12]:
class PartialCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(PartialCrossEntropyLoss, self).__init__()

    def forward(self, predictions, targets, mask):
        focal_loss = F.cross_entropy(predictions, targets, reduction='none')
        masked_focal_loss = focal_loss * mask
        pfce_loss = masked_focal_loss.sum() / mask.sum()
        return pfce_loss

In [13]:
# Initialize model, loss function, and optimizer
model = deeplabv3_resnet101(pretrained=False, progress=False, num_classes=2)
loss_fn = PartialCrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# **Training and Evaluation**

In [14]:
# Training loop
# Reduced number of epochs from 3 to 1
for epoch in range(1):
    # Decreased batch size in the dataloader from 4 to 2
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    for images, masks, point_masks in dataloader:
        optimizer.zero_grad()
        outputs = model(images)['out']
        # Ensure targets (masks) are in shape (batch_size, height, width)
        masks = masks.squeeze(1)
        point_masks = point_masks.squeeze(1)
        loss = loss_fn(outputs, masks, point_masks)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/7906_sat_66.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/7906_mask_66.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/13415_sat_48.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/13415_mask_48.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/855_sat_42.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/855_mask_42.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/15573_sat_44.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/15573_mask_44.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/7906_sat_84.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/7906_mask_84.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/33573_sat_24.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/33573_mask_2

# **Experiments with different parameters**

In [ ]:
# Function to train the model with different parameters
def train_model(learning_rate, batch_size, num_epochs=3):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    model = deeplabv3_resnet101(pretrained=False, progress=False, num_classes=2)
    loss_fn = PartialCrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for images, masks, point_masks in dataloader:
            optimizer.zero_grad()
            outputs = model(images)['out']
            masks = masks.squeeze(1)
            point_masks = point_masks.squeeze(1)
            loss = loss_fn(outputs, masks, point_masks)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}')
    return epoch_loss / len(dataloader)

# Experiment with different learning rates and batch sizes
learning_rates = [0.0001, 0.00001]
batch_sizes = [4, 8]

results = {}
for lr in learning_rates:
    for bs in batch_sizes:
        print(f'Training with learning rate: {lr} and batch size: {bs}')
        avg_loss = train_model(learning_rate=lr, batch_size=bs)
        results[(lr, bs)] = avg_loss

print('Results:')
for (lr, bs), avg_loss in results.items():
    print(f'Learning Rate: {lr}, Batch Size: {bs}, Average Loss: {avg_loss}')

Training with learning rate: 0.0001 and batch size: 4


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Streaming output truncated to the last 5000 lines.
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/7906_mask_55.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/3484_sat_58.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/3484_mask_58.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/13415_sat_35.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/13415_mask_35.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/20187_sat_14.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/20187_mask_14.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/13415_sat_58.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/13415_mask_58.jpg
Image path: /content/drive/MyDrive/Forest Aerial Segmentation/image/15573_sat_31.jpg
Mask path: /content/drive/MyDrive/Forest Aerial Segmentation/mask/15573_mask_31.jpg
Image path: /content/dr